In [ ]:
import numpy as np
import random
import tensorflow as tf
import sklearn.datasets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
DIM = 256
CRITIC_ITERS = 1
BATCH_SIZE = 256
ITERS = 100000
DATASET = '25gaussians'
num_classes = 5

In [ ]:
def spectral_norm(w, iteration=1):
    w_shape = w.shape.as_list()
    w = tf.reshape(w, [-1, w_shape[-1]])

    u = tf.get_variable('u', [1, w_shape[-1]], initializer=tf.random_normal_initializer(), trainable=False)

    u_hat = u
    v_hat = None
    for i in range(iteration):
        v_ = tf.matmul(u_hat, tf.transpose(w))
        v_hat = tf.nn.l2_normalize(v_)

        u_ = tf.matmul(v_hat, w)
        u_hat = tf.nn.l2_normalize(u_)

    u_hat = tf.stop_gradient(u_hat)
    v_hat = tf.stop_gradient(v_hat)

    sigma = tf.matmul(tf.matmul(v_hat, w), tf.transpose(u_hat))

    with tf.control_dependencies([u.assign(u_hat)]):
        w_norm = w / sigma
        w_norm = tf.reshape(w_norm, w_shape)

    return w_norm

def snlinear(inputs, input_dim, output_dim, activation, name):
    with tf.variable_scope(name):
        weights = tf.get_variable(
            'kernel',
            [input_dim, output_dim],
            initializer=tf.random_normal_initializer(0., output_dim ** -0.5)
        )
        biases = tf.get_variable('bias', [output_dim])
        outputs = tf.matmul(inputs, spectral_norm(weights)) + biases
        if activation:
            outputs = activation(outputs)

        return outputs

def Generator(noise, condition):
    with tf.variable_scope('Generator', reuse=tf.AUTO_REUSE):
        inputs = tf.concat([condition, noise], axis=-1)
        outputs = tf.layers.dense(inputs, DIM, activation=tf.nn.relu, name='Generator.1')
        outputs = tf.layers.dense(outputs, DIM, activation=tf.nn.relu, name='Generator.2')
        outputs = tf.layers.dense(outputs, DIM, activation=tf.nn.relu, name='Generator.3')
        outputs = tf.layers.dense(outputs, 2, activation=None, name='Generator.4')
        return outputs

def Discriminator(inputs):
    with tf.variable_scope('Discriminator', reuse=tf.AUTO_REUSE):
        outputs = snlinear(inputs, 2, DIM, activation=tf.nn.relu, name='Discriminator.1')
        outputs = snlinear(outputs, DIM, DIM, activation=tf.nn.relu, name='Discriminator.2')
        outputs = snlinear(outputs, DIM, DIM, activation=tf.nn.relu, name='Discriminator.3')
        outputs = snlinear(outputs, DIM, 1, activation=None, name='Discriminator.4')
        return outputs

def Auxilliary(inputs):
    with tf.variable_scope('Auxilliary', reuse=tf.AUTO_REUSE):
        outputs = tf.layers.dense(inputs, DIM, activation=tf.nn.relu, name='Auxilliary.1')
        outputs = tf.layers.dense(outputs, num_classes, activation=tf.nn.softmax, name='Auxilliary.2')
        return outputs

In [ ]:
real_data = tf.placeholder(tf.float32, shape=[None, 2])
noise_data = tf.placeholder(tf.float32, shape=[None, 2])
real_condition = tf.placeholder(tf.float32, shape=[None, 5])
fake_condition = tf.placeholder(tf.float32, shape=[None, 5])

fake_data = Generator(noise_data, fake_condition)
disc_real = Discriminator(real_data)
disc_fake = Discriminator(fake_data)
auxilliary_real = Auxilliary(real_data)
auxilliary_fake = Auxilliary(fake_data)

disc_loss = tf.reduce_mean(disc_fake) - tf.reduce_mean(disc_real)
gen_loss = -tf.reduce_mean(disc_fake)
aux_real_loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(logits=auxilliary_real, labels=real_condition))
aux_fake_loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(logits=auxilliary_fake, labels=fake_condition))
aux_loss = aux_real_loss + aux_fake_loss

In [ ]:
disc_params = tf.trainable_variables('Discriminator')
gen_params = tf.trainable_variables('Generator')
aux_params = tf.trainable_variables('Auxilliary')

disc_train_op = tf.train.AdamOptimizer(
    learning_rate=5e-4,
    beta1=0.0,
    beta2=0.9
).minimize(disc_loss, var_list=disc_params)

gen_train_op = tf.train.AdamOptimizer(
    learning_rate=1e-4,
    beta1=0.0,
    beta2=0.9
).minimize(gen_loss + aux_loss, var_list=gen_params + aux_params)

In [ ]:
def create_real_data():
    positions = np.random.choice(range(-2, 3), size=BATCH_SIZE*2)
    condition = np.identity(5)[np.reshape(positions, [BATCH_SIZE, 2])[:, 0] + 2]
    noise = 0.05 * np.random.randn(BATCH_SIZE*2)
    yield np.reshape(2 * positions + noise, [BATCH_SIZE, 2]) / 2.828, condition

def create_noise_data():
    fake_data = np.random.uniform(-1, 1, size=[BATCH_SIZE, 2])
    condition = np.random.choice(range(0, 5), size=BATCH_SIZE)
    condition = np.identity(5)[condition]
    yield fake_data, condition.astype(np.float32)

def generate_image(condition):
    real_data_samples, _ = next(create_real_data())
    noise_data_samples, _ = next(create_noise_data())
    fake_data_samples = sess.run(fake_data, {noise_data: noise_data_samples, fake_condition: condition})

    plt.scatter(real_data_samples[:, 0], real_data_samples[:, 1], c='orange', marker='+')
    plt.scatter(fake_data_samples[:, 0], fake_data_samples[:, 1], c='green', marker='+')
    plt.show()

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
%%time

for i in range(ITERS+1):
    real_data_batch, real_condition_batch = next(create_real_data())
    noise_data_batch, fake_condition_batch = next(create_noise_data())
    fd = {
        real_data: real_data_batch,
        noise_data: noise_data_batch,
        real_condition: real_condition_batch,
        fake_condition: fake_condition_batch
    }
    for _ in range(CRITIC_ITERS):
        _disc_loss, _ = sess.run([disc_loss, disc_train_op], fd)

    _, _gen_loss, _aux_loss = sess.run([gen_train_op, gen_loss, aux_loss], fd)

    if i % 10000 == 0:
        print(i, 'D_loss: {:.5f}, G_loss: {:.5f}, Q_loss: {:.5f}'.format(_disc_loss, _gen_loss, _aux_loss))
        _, fake_condition_batch = next(create_noise_data())
        generate_image(fake_condition_batch)


In [ ]:
generate_image([[0, 0, 1, 0, 0]] * BATCH_SIZE)